In [1]:
from numpy import mean
from numpy import std
import numpy as np
import os
import pandas as pd
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
import networkx as nx

In [21]:
NUMBER_NODES = 9

def load_data():
    train_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_train.csv'))
    val_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_val.csv'))
    test_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_test.csv'))

    featuresNumber = (NUMBER_NODES * NUMBER_NODES - NUMBER_NODES) // 2 
    def get_tuple_tensor_dataset(row):
        X = row[0 : featuresNumber].astype('float32')
        Y = row[featuresNumber + 1: ].astype('float32') # Inclui a banda otima na posicao 0
        return X, Y

    train_dataset = list(map(get_tuple_tensor_dataset, train_df.to_numpy()))
    val_dataset = list(map(get_tuple_tensor_dataset, val_df.to_numpy()))
    test_dataset = list(map(get_tuple_tensor_dataset, test_df.to_numpy()))

    X = []
    Y = []
    for x, y in train_dataset:
        X.append(x)
        Y.append(y)
    x_train = np.array(X, dtype=object)
    y_train = np.array(Y, dtype=object)

    X = []
    Y = []
    for x, y in test_dataset:
        X.append(x)
        Y.append(y)
    x_test = np.array(X, dtype=object)
    y_test = np.array(Y, dtype=object)

    X = []
    Y = []
    for x, y in val_dataset:
        X.append(x)
        Y.append(y)
    x_val = np.array(X)
    y_val = np.array(Y)

    x_train = np.concatenate((x_train, x_val))
    y_train = np.concatenate((y_train, y_val))


    return x_train, y_train, x_test, y_test

In [22]:
x_train, y_train, x_test, y_test = load_data()

In [23]:
from sklearn.tree import DecisionTreeRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=2652124)
n_scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
n_scores = absolute(n_scores)
print('MAE Error: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# define the chained multioutput wrapper model
# wrapper = RegressorChain(model)

MAE Error: 1.539 (0.012)


In [24]:
model.fit(x_train, y_train)

DecisionTreeRegressor()

In [25]:
print(model.get_depth())

32


In [26]:
def count_repeats(output):
    counts = np.unique(np.round(output))
    repeated = NUMBER_NODES - counts.shape[0]
    return repeated

def get_valid_pred(pred):
    valid = np.ones(NUMBER_NODES)
    labels = np.arange(0, NUMBER_NODES)
    for i in labels:
        min_value = np.amin(pred)
        min_idx, = np.where(pred == min_value)
        min_idx = min_idx[0]
        pred[min_idx] = 100
        valid[min_idx] = i
    return valid
    
def get_bandwidth(Graph, nodelist):
    Graph = nx.Graph(Graph)
    if not Graph.edges:
        return 0
    if nodelist.all() != None:
        L = nx.laplacian_matrix(Graph, nodelist=nodelist)
    else:
        L = nx.laplacian_matrix(Graph)
    x, y = np.nonzero(L)
    return (x-y).max()

def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj

In [27]:
pred = model.predict(x_test)

sumTest_original = 0
sumTest_pred = 0
sumTest_true = 0

count = 0
cases_with_repetition = 0

for i in range(len(pred)):

    output = pred[i]

    quantity_repeated = count_repeats(np.round(output))
    print('Pred: ', output)
    print('True: ', y_test[i])
    if quantity_repeated != 0:
        cases_with_repetition += 1
    output = get_valid_pred(output)
    print('Pred valid: ', output)
    count += quantity_repeated

    print("Bandwidth")
    graph = getGraph(x_test[i])
    original_band = get_bandwidth(graph, np.array(None))
    sumTest_original += original_band
    pred_band = get_bandwidth(graph, output)
    sumTest_pred += pred_band
    true_band = get_bandwidth(graph, y_test[i])
    sumTest_true += true_band
    print("Bandwidth")
    print(original_band)
    print(pred_band)
    print(true_band)
print('Quantidade de rótulos repetidos, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 6 - ', count)
print('Quantidade de saídas com repetição, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 1 - ', cases_with_repetition)
test_length = pred.shape[0]
print('Test length - ', test_length)
print("Bandwidth mean")
print(sumTest_original / test_length)
print("Pred bandwidth mean")
print(sumTest_pred / test_length)
print("True bandwidth mean")
print(sumTest_true / test_length)

Pred:  [1. 8. 0. 7. 3. 4. 6. 2. 5.]
True:  [1.0 8.0 4.0 5.0 6.0 7.0 0.0 2.0 3.0]
Pred valid:  [1. 8. 0. 7. 3. 4. 6. 2. 5.]
Bandwidth
Bandwidth
7
1
1
Pred:  [6. 0. 1. 7. 8. 2. 3. 4. 5.]
True:  [1.0 8.0 0.0 7.0 4.0 6.0 5.0 2.0 3.0]
Pred valid:  [6. 0. 1. 7. 8. 2. 3. 4. 5.]
Bandwidth
Bandwidth
8
3
2
Pred:  [6. 1. 0. 8. 7. 2. 3. 5. 4.]
True:  [1.0 0.0 8.0 7.0 2.0 5.0 3.0 6.0 4.0]
Pred valid:  [6. 1. 0. 8. 7. 2. 3. 5. 4.]
Bandwidth
Bandwidth
8
2
2
Pred:  [3. 4. 8. 1. 2. 0. 6. 7. 5.]
True:  [6.0 5.0 7.0 4.0 3.0 0.0 8.0 1.0 2.0]
Pred valid:  [3. 4. 8. 1. 2. 0. 6. 7. 5.]
Bandwidth
Bandwidth
8
3
3
Pred:  [4. 5. 2. 3. 1. 8. 6. 7. 0.]
True:  [7.0 1.0 8.0 0.0 2.0 3.0 4.0 6.0 5.0]
Pred valid:  [4. 5. 2. 3. 1. 8. 6. 7. 0.]
Bandwidth
Bandwidth
8
3
3
Pred:  [1. 2. 7. 0. 8. 4. 3. 5. 6.]
True:  [7.0 1.0 0.0 8.0 3.0 4.0 2.0 6.0 5.0]
Pred valid:  [1. 2. 7. 0. 8. 4. 3. 5. 6.]
Bandwidth
Bandwidth
8
3
3
Pred:  [5. 2. 4. 1. 8. 6. 0. 7. 3.]
True:  [3.0 6.0 2.0 5.0 8.0 0.0 4.0 1.0 7.0]
Pred valid:  [5. 2. 4. 1.

# Notes

## No Chained Multioutput Regression
- Only DecisionTreeRegressor, counted repeats: 0
- Only LinearRegression, counted repeats: 238
- Only KNeighborsRegressor, counted repeats: I couldn't get the output (infinite loading)

## Chained Multioutput Regression

- DecisionTreeRegressor, counted repeats: 26, 30
- LinearRegression, counted repeats: 238
- KNeighborsRegressor, counted repeats: 119
- SVM, actually support vector regression: 213

## No Chained Multioutput Regression but treating eache output independently (wrong)
- SVM, actually support vector regression: 194
- KNeighborsRegressor, counted repeats: I couldn't get the output (infinite loading)
- LinearRegression, counted repeats: 238
- DecisionTreeRegressor, counted repeats: 116

### https://machinelearningmastery.com/multi-output-regression-models-with-python/

#### Essa abordagens eu testei:
https://machinelearningmastery.com/multi-label-classification-with-deep-learning/
(sigmoid no final, e binary_crossentropy como loss)

https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-satellite-photos-of-the-amazon-rainforest/